In [1]:
#import libraries 
import pandas as pd
import requests
import json
import datetime 
import sqlalchemy
import random
import numpy as np
from datetime import datetime, date, timedelta
from pytz import timezone

In [5]:
import pandas as pd
import requests
import json
import datetime 
import sqlalchemy
import random
import numpy as np
from datetime import datetime, date, timedelta
from pytz import timezone

#create function
def get_flights(icao_list):

    today = datetime.now().date()
    tomorrow = (today + timedelta(days=1))
    
    flights_data = []
    flights_json = {} 

    #create for loop for each ICAO
    for icao in icao_list:
        times = [["00:00", "11:59"], ["12:00", "23:59"]]
        
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg": "true", "direction": "Arrival", "withCancelled": "false", "withCodeshared": "true", "withCargo": "false", "withPrivate": "false"}
            flight_headers = {
                "X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers=flight_headers, params=querystring)
            print(response)
            flights_json = response.json()
            
            for flight in flights_json['arrivals']:
                flights_dict = {}
                flights_dict['arrival_icao'] = icao
                flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
                flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
                flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
                flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
                flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
                flights_dict['airline'] = flight['airline'].get('name', None)
                flights_dict['flight_number'] = flight.get('number', None)
                flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
                flights_data.append(flights_dict)
    
    return pd.DataFrame(flights_data)

icaos = ['EGLL','EGKK','EGGW','EGSS','EGMC','EDDB','EDDH','EGCC','LEBL']
get_flights(icaos)
flights_data = get_flights(icaos)

In [6]:
icaos = ['EGLL','EGKK','EGGW','EGSS','EGMC','EDDB','EDDH','EGCC','LEBL']
get_flights(icaos)
flights_data = get_flights(icaos)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [7]:
flights_data

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EGLL,2023-09-24 05:45+01:00,3,Manama,OBBI,None,American,AA 6442,2023-09-23
1,EGLL,2023-09-24 05:30+01:00,5,Jo'anna,FAOR,None,American,AA 6446,2023-09-23
2,EGLL,2023-09-24 05:50+01:00,5,Cape Town,FACT,None,American,AA 6448,2023-09-23
3,EGLL,2023-09-24 05:50+01:00,5,Nairobi,HKJK,None,American,AA 6654,2023-09-23
4,EGLL,2023-09-24 05:30+01:00,5,Singapore,WSSS,2023-09-23 22:35+08:00,American,AA 6874,2023-09-23
...,...,...,...,...,...,...,...,...,...
5002,LEBL,2023-09-24 23:45+02:00,1,Paris,LFPG,2023-09-24 21:55+02:00,Vueling,VY 8241,2023-09-23
5003,LEBL,2023-09-24 23:40+02:00,1,Amsterdam,EHAM,2023-09-24 21:20+02:00,Vueling,VY 8305,2023-09-23
5004,LEBL,2023-09-24 23:55+02:00,1,Lisbon,LPPT,2023-09-24 21:00+01:00,Vueling,VY 8463,2023-09-23
5005,LEBL,2023-09-24 23:10+02:00,2,Brussels,EBBR,2023-09-24 21:05+02:00,Ryanair,FR 2921,2023-09-23


In [ ]:
flights_data

In [8]:
#create connection to MySQL
schema="gans"
host="127.0.0.1"
user="root"
password="LexiHarley0323"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [9]:
#create the flights_data table in MySQL
flights_data.to_sql('flights', 
              if_exists='append', 
              con=con, 
              index=False)

5007